In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bulgan,MN,48.8125,103.5347,57.42,80,96,1.34,overcast clouds
1,1,Saint Anthony,US,45.0205,-93.2180,80.91,68,0,17.27,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,78.96,94,75,5.75,broken clouds
3,3,Carnarvon,AU,-24.8667,113.6333,67.96,38,0,7.47,clear sky
4,4,Khudumelapye,BW,-23.8833,24.7500,52.92,28,6,5.59,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saint Anthony,US,45.0205,-93.2180,80.91,68,0,17.27,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,78.96,94,75,5.75,broken clouds
6,6,Viransehir,TR,37.2353,39.7631,88.61,33,0,5.30,clear sky
14,14,Butaritari,KI,3.0707,172.7902,82.87,73,84,11.79,broken clouds
21,21,Tupelo,US,34.2576,-88.7034,82.69,78,0,5.75,clear sky
26,26,San Juan De Uraba,CO,8.7592,-76.5297,76.15,91,94,6.76,overcast clouds
28,28,Pandan,PH,14.0453,124.1694,86.31,65,100,9.22,moderate rain
33,33,Merauke,ID,-8.4667,140.3333,80.58,84,98,14.74,overcast clouds
34,34,Atuona,PF,-9.8000,-139.0333,77.74,71,12,15.70,few clouds
46,46,Tuy Hoa,VN,13.0833,109.3000,88.92,56,100,13.47,overcast clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saint Anthony,US,45.0205,-93.2180,80.91,68,0,17.27,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,78.96,94,75,5.75,broken clouds
6,6,Viransehir,TR,37.2353,39.7631,88.61,33,0,5.30,clear sky
14,14,Butaritari,KI,3.0707,172.7902,82.87,73,84,11.79,broken clouds
21,21,Tupelo,US,34.2576,-88.7034,82.69,78,0,5.75,clear sky
...,...,...,...,...,...,...,...,...,...,...
655,655,Shirokiy,RU,49.7580,129.5230,75.22,67,79,11.23,broken clouds
657,657,Quang Ngai,VN,15.1167,108.8000,89.49,58,97,4.65,overcast clouds
662,662,Banjar,ID,-8.1900,114.9675,88.61,48,55,1.79,broken clouds
663,663,Moussoro,TD,13.6408,16.4901,77.95,77,100,10.85,light rain


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint Anthony,US,80.91,clear sky,45.0205,-93.2180,
2,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
6,Viransehir,TR,88.61,clear sky,37.2353,39.7631,
14,Butaritari,KI,82.87,broken clouds,3.0707,172.7902,
21,Tupelo,US,82.69,clear sky,34.2576,-88.7034,
26,San Juan De Uraba,CO,76.15,overcast clouds,8.7592,-76.5297,
28,Pandan,PH,86.31,moderate rain,14.0453,124.1694,
33,Merauke,ID,80.58,overcast clouds,-8.4667,140.3333,
34,Atuona,PF,77.74,few clouds,-9.8000,-139.0333,
46,Tuy Hoa,VN,88.92,overcast clouds,13.0833,109.3000,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.loc[(hotel_df["Hotel Name"]!="")]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint Anthony,US,80.91,clear sky,45.0205,-93.2180,Holiday Inn Express Roseville-St. Paul
2,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Viransehir,TR,88.61,clear sky,37.2353,39.7631,Viranşehir Öğretmenevi
14,Butaritari,KI,82.87,broken clouds,3.0707,172.7902,Isles Sunset Lodge
21,Tupelo,US,82.69,clear sky,34.2576,-88.7034,"Hilton Garden Inn Tupelo, MS"
26,San Juan De Uraba,CO,76.15,overcast clouds,8.7592,-76.5297,Hotel Cerro De Zumba
28,Pandan,PH,86.31,moderate rain,14.0453,124.1694,Bonifacio Lodge
34,Atuona,PF,77.74,few clouds,-9.8000,-139.0333,Villa Enata
46,Tuy Hoa,VN,88.92,overcast clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
52,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn


In [14]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template =  """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Holiday Inn Express Roseville-St. Paul</dd>\n<dt>City</dt><dd>Saint Anthony</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>80.91 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Max Temp</dt><dd>78.96 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Viranşehir Öğretmenevi</dd>\n<dt>City</dt><dd>Viransehir</dd>\n<dt>Country</dt><dd>TR</dd>\n<dt>Max Temp</dt><dd>88.61 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Isles Sunset Lodge</dd>\n<dt>City</dt><dd>Butaritari</dd>\n<dt>Country</dt><dd>KI</dd>\n<dt>Max Temp</dt><dd>82.87 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hilton Garden Inn Tupelo, MS</dd>\n<dt>City</dt><dd>Tupelo</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>82.69 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Cerro De Zumba</dd>\n<dt>City</dt><dd>San Juan De Uraba</dd>\n<dt>Country</dt><dd>CO</dd>\n<dt>Ma

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))